# Setup

In [10]:
%load_ext autoreload
%autoreload 2

from crispr import Spatial
import scanpy as sc
import squidpy as sq
import functools
import os
from os import system
import pandas as pd

ddd = os.path.join(os.path.expanduser("~"), "projects/data/public/xenium")

pd.options.display.max_columns = 100

# genes = ["CDKN1A", "CDKN2A", "TP53", "PLAUR",
#          "PTGER4", "ATG16L1", "NOD2", "LACC1"]
# cell_types = None
# palette = "tab20"

genes = dict(CSF=["CSF2", "CSF2RB", "CSF2RA"],
             All_ILCs=["IL7R"],
             ILC1=["TBX21", "KLRB1", "CCL5", "HSP90AA1", "IFNG"],
             ILC3=["RORC", "IL23R", "KIT", "CCL20", "TCF7", "IL22"],
             Cytokines=["IFNG", "IL22"])  # ILC1 & ILC3, respectively
cell_types = ["ILC3", "LTi-like NCR+ ILC3", "LTi-like NCR- ILC3",
              "ILCP", "Macrophages", "Stem cells"]
# palette = ["r", "tab:pink", "m", "b", "tab:brown", "cyan"]
palette = "tab10"

kws_pp = dict(cell_filter_pmt=[0, 100], cell_filter_ncounts=None,
              cell_filter_ngene=[0, None], gene_filter_ncell=[3, None],
              gene_filter_ncounts=None)  # can call using 1 arguments

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

In [11]:
d_p = os.path.join(
    ddd, "Xenium_V1_hColon_Non_diseased_Base_FFPE_outs/cell_feature_matrix")
self = Spatial(d_p)



<<< INITIALIZING SPATIAL CLASS OBJECT >>>



<<< INITIALIZING OMICS CLASS OBJECT >>>

col_gene_symbols="gene_symbols"
col_cell_type="leiden"
col_sample_id=None
col_batch=None
col_subject=None
col_condition=None
col_num_umis=None
key_control=None
key_treatment=None


<<< LOADING 10X FILE /home/elizabeth/elizabeth/projects/data/public/xenium/Xenium_V1_hColon_Non_diseased_Base_FFPE_outs/cell_feature_matrix >>>


Cell Counts: Initial


Observations: 270984



Gene Counts: Initial

Empty DataFrame
Columns: [index, gene_ids, feature_types]
Index: []





 AnnData object with n_obs × n_vars = 270984 × 541
    var: 'gene_ids', 'feature_types'
    layers: 'counts'


 AnnData object with n_obs × n_vars = 270984 × 541
    var: 'gene_ids', 'feature_types'
    layers: 'counts'
                     gene_ids    feature_types
gene_symbols                                  
ABCC8         ENSG00000006071  Gene Expression
ADH1C         ENSG00000248144  Gene Expression
ADRA2A        ENSG00000150594  Gene

## Preprocessing

In [12]:
%%time

_ = self.preprocess(**kws_pp)


Un-Used Keyword Arguments: {'assay_protein': None, 'col_sample_id': None, 'col_batch': None, 'col_subject': None, 'col_condition': None, 'col_num_umis': None}




## Clustering

In [ ]:
%%time

_ = self.cluster()

# Centrality Scores

In [ ]:
%%time

self.calculate_centrality(col_cell_type="majority_voting", n_jobs=4)

# Cell Type Co-Occurrence

In [ ]:
%%time

_ = self.find_cooccurrence(key_cell_type=cell_types, figsize=(60, 20),
                           palette=palette, kws_plot=dict(wspace=3))

# Neighbors Enrichment Analysis

In [ ]:
%%time

_ = self.calculate_neighborhood(figsize=(60, 30), cmap="coolwarm",
                                cbar_range=[-30, 30])

# Spatially-Variable Genes

In [ ]:
%%time

ggg = genes if genes and isinstance(genes, dict) else {"Genes": genes}
genes_all = functools.reduce(lambda i, j: i + j, [ggg[g] for g in ggg])
# for g in ggg:
#   self.find_svgs(genes=ggg[g])
fig = self.find_svgs(
    genes=genes_all, method="moran", n_perms=10, kws_plot=dict(
        wspace=0.1, cmap="coolwarm", vcenter=0, size=3,
        legend_fontsize="large"), figsize=(15, 15))

# Receptor-Ligand Interactions

In [ ]:
%%time

out = self.calculate_receptor_ligand(
    key_sources=cell_types, key_targets=cell_types, col_condition=False,
    p_threshold=0.001, remove_ns=True, figsize=(15, 10))